In [1]:
import psycopg2

In [2]:
# attempting to connect with no credentials
conn = psycopg2.connect()

TypeError: missing dsn and no parameters

In [3]:
# attempting to connect to the main postres db, then create a new one from there
conn = psycopg2.connect(dbname="postgres")

In [4]:
conn.autocommit = True

In [5]:
cursor = conn.cursor()

In [6]:
CREATE_DATABASE_QUERY = "CREATE DATABASE housing_data;"

In [7]:
cursor.execute(CREATE_DATABASE_QUERY)

In [8]:
conn.close()

In [9]:
# that all seemed to work, moving on to making a schema for the sales info

# actually it seems tedious/slow to insert each row individually.  and it seems like I can avoid that if I
# am willing to use ALL of the columns, rather than making a bespoke schema for my purposes

In [10]:
conn = psycopg2.connect(dbname="housing_data")

In [11]:
cursor = conn.cursor()

In [12]:
COPY_SALES_QUERY = """COPY sales FROM STDIN WITH (FORMAT CSV)"""

In [13]:
sales_zip_file, sales_csv_file = collect_sales_data()

NameError: name 'collect_sales_data' is not defined

In [14]:
from data_collection import collect_sales_data

In [24]:
sales_zip_file, sales_csv_file = collect_sales_data()

In [16]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

UndefinedTable: relation "sales" does not exist


In [17]:
# ok, annoying, none of the docs indicate that you have to make the table before copying the CSV

In [18]:
CREATE_SALES_TABLE_QUERY = """
DROP TABLE IF EXISTS sales;
CREATE TABLE sales (
  id                 SERIAL PRIMARY KEY,
  ExciseTaxNbr       INT,
  Major              CHAR(6),
  Minor              CHAR(4),
  DocumentDate       DATE,
  SalePrice          INT,
  RecordingNbr       CHAR(14),
  Volume             CHAR(3),
  Page               CHAR(3),
  PlatNbr            CHAR(6),
  PlatType           CHAR(1),
  PlatLot            CHAR(14),
  PlatBlock          CHAR(7),
  SellerName         TEXT,
  BuyerName          TEXT,
  PropertyType       INT,
  PrincipalUse       INT,
  SaleInstrument     INT,
  AFForestLand       CHAR(1),
  AFCurrentUseLand   CHAR(1),
  AFNonProfitUse     CHAR(1),
  AFHistoricProperty CHAR(1),
  SaleReason         INT,
  PropertyClass      INT,
  SaleWarning        TEXT
);
"""

In [19]:
cursor.execute(CREATE_SALES_TABLE_QUERY)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [20]:
conn.close()

In [21]:
conn = psycopg2.connect(dbname="housing_data")

In [22]:
cursor = conn.cursor()

In [23]:
cursor.execute(CREATE_SALES_TABLE_QUERY)

In [25]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

InvalidTextRepresentation: invalid input syntax for integer: "ExciseTaxNbr"
CONTEXT:  COPY sales, line 1, column id: "ExciseTaxNbr"


In [26]:
sales_zip_file, sales_csv_file = collect_sales_data()

In [27]:
next(sales_csv_file) # Skip the header row

'"ExciseTaxNbr","Major","Minor","DocumentDate","SalePrice","RecordingNbr","Volume","Page","PlatNbr","PlatType","PlatLot","PlatBlock","SellerName","BuyerName","PropertyType","PrincipalUse","SaleInstrument","AFForestLand","AFCurrentUseLand","AFNonProfitUse","AFHistoricProperty","SaleReason","PropertyClass","SaleWarning"\n'

In [28]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [29]:
conn.close()
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()

In [30]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

UndefinedTable: relation "sales" does not exist


In [31]:
cursor.execute(CREATE_SALES_TABLE_QUERY)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [32]:
conn.close()
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()

In [33]:
cursor.execute(CREATE_SALES_TABLE_QUERY)

In [34]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

StringDataRightTruncation: value too long for type character(4)
CONTEXT:  COPY sales, line 1, column minor: "03/19/1998"


In [35]:
next(sales_csv_file)

StopIteration: 

In [37]:
# huh, looks like I can't have an id column

In [38]:
CREATE_SALES_TABLE_QUERY = """
DROP TABLE IF EXISTS sales;
CREATE TABLE sales (
  ExciseTaxNbr       INT,
  Major              CHAR(6),
  Minor              CHAR(4),
  DocumentDate       DATE,
  SalePrice          INT,
  RecordingNbr       CHAR(14),
  Volume             CHAR(3),
  Page               CHAR(3),
  PlatNbr            CHAR(6),
  PlatType           CHAR(1),
  PlatLot            CHAR(14),
  PlatBlock          CHAR(7),
  SellerName         TEXT,
  BuyerName          TEXT,
  PropertyType       INT,
  PrincipalUse       INT,
  SaleInstrument     INT,
  AFForestLand       CHAR(1),
  AFCurrentUseLand   CHAR(1),
  AFNonProfitUse     CHAR(1),
  AFHistoricProperty CHAR(1),
  SaleReason         INT,
  PropertyClass      INT,
  SaleWarning        TEXT
);
"""

In [39]:
conn.close()
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()

In [40]:
cursor.execute(CREATE_SALES_TABLE_QUERY)

In [41]:
sales_zip_file, sales_csv_file = collect_sales_data()
next(sales_csv_file) # Skip the header row

'"ExciseTaxNbr","Major","Minor","DocumentDate","SalePrice","RecordingNbr","Volume","Page","PlatNbr","PlatType","PlatLot","PlatBlock","SellerName","BuyerName","PropertyType","PrincipalUse","SaleInstrument","AFForestLand","AFCurrentUseLand","AFNonProfitUse","AFHistoricProperty","SaleReason","PropertyClass","SaleWarning"\n'

In [42]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

In [43]:
conn.commit()

In [44]:
# run through everything beginning to end
from data_collection import collect_sales_data
sales_zip_file, sales_csv_file = collect_sales_data()

In [45]:
from sql_utils import create_database, create_sales_table, copy_csv_to_sales_table
create_database()

ActiveSqlTransaction: DROP DATABASE cannot run inside a transaction block


In [46]:
%load_ext autoreload

In [47]:
%autoreload 2

In [48]:
create_database()

ActiveSqlTransaction: DROP DATABASE cannot run inside a transaction block


In [49]:
create_database()

ObjectInUse: database "housing_data" is being accessed by other users
DETAIL:  There is 1 other session using the database.


In [50]:
conn.close()

In [51]:
create_database()

In [52]:
# try again to run everything end-to-end

In [53]:
from data_collection import collect_sales_data
sales_zip_file, sales_csv_file = collect_sales_data()

In [54]:
from sql_utils import create_database, create_sales_table, copy_csv_to_sales_table
create_database()

In [55]:
conn = psycopg2.connect(dbname="housing_data")

In [56]:
create_sales_table(conn)

In [57]:
copy_csv_to_sales_table(conn, sales_csv_file)

In [58]:
sales_zip_file.close()
sales_csv_file.close()
conn.close()

In [59]:
# okay, that all ran, let's see if we can read stuff!
conn = psycopg2.connect(dbname="housing_data")

In [60]:
import pandas as pd

In [61]:
pd.read_sql_query("SELECT * FROM sales LIMIT 5;", conn)

,excisetaxnbr,major,minor,documentdate,saleprice,recordingnbr,volume,page,platnbr,plattype,...,propertytype,principaluse,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning
0,1600768,330405,0100,1998-03-19,215000,199803251689,145,039,330405,C,...,2,2,3,N,N,N,N,1,3,
1,2413752,868146,0030,2009-09-11,0,20091022001461,,,,,...,3,2,15,N,N,N,N,11,3,18 31 38
2,1939480,258190,0265,2003-02-06,0,20030214003390,,,,,...,3,6,15,N,N,N,N,10,8,31 51
3,2999169,919715,0200,2019-07-08,192000,20190712001080,,,,,...,3,2,3,N,N,N,N,1,3,
4,2220242,334330,1343,2006-05-30,0,20060706002163,,,,,...,1,6,3,N,N,N,N,18,7,11 31


In [62]:
pd.read_sql_query("SELECT * FROM sales WHERE date_part('year', DocumentDate) = 2018 LIMIT 5;", conn)

,excisetaxnbr,major,minor,documentdate,saleprice,recordingnbr,volume,page,platnbr,plattype,...,propertytype,principaluse,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning
0,2947815,683782,0300,2018-07-23,280001,20180816000825,,,,,...,0,6,21,N,N,N,N,1,8,14 31
1,2937935,873178,0100,2018-06-20,185000,20180625000560,,,,,...,14,2,3,N,N,N,N,1,3,
2,2912844,387610,0640,2018-01-24,0,,,,,,...,3,6,15,N,N,N,N,5,8,12 18 31
3,2952652,281809,0020,2018-09-13,165000,20180914001119,,,,,...,3,2,3,N,N,N,N,1,3,15
4,2936365,260779,0010,2018-06-07,650000,20180615001343,,,,,...,3,2,3,N,N,N,N,1,3,


In [63]:
# ok, this is working, let's make another commit

In [64]:
# time for buildings and parcels!

In [65]:
CREATE_BUILDINGS_TABLE_QUERY = """
    DROP TABLE IF EXISTS buildings;
    CREATE TABLE BUILDINGS (
        Major              CHAR(6),
        Minor              CHAR(4),
        BldgNbr            INT,
        NbrLivingUnits     INT,
        Address            TEXT,
        BuildingNumber     CHAR(5),
        Fraction           CHAR(3),
        DirectionPrefix    CHAR(2),
        StreetName         CHAR(25),
        StreetType         CHAR(6),
        DirectionSuffix    CHAR(2),
        ZipCode            CHAR(10),
        Stories            INT,
        BldgGrade          INT,
        BldgGradeVar       INT,
        SqFt1stFloor       INT,
        SqFtHalfFloor      INT,
        SqFt2ndFloor       INT,
        SqFtUpperFloor     INT,
        SqFtUnfinFull      INT,
        SqFtUnfinHalf      INT,
        SqFtTotLiving      INT,
        SqFtTotBasement    INT,
        SqFtFinBasement    INT,
        FinBasementGrade   INT,
        SqFtGarageBasement INT,
        SqFtGarageAttached INT,
        DaylightBasement   CHAR(1),
        SqFtOpenPorch      INT,
        SqFtEnclosedPorch  INT,
        SqFtDeck           INT,
        HeatSystem         INT,
        HeatSource         INT,
        BrickStone         INT,
        ViewUtilization    CHAR(1),
        Bedrooms           INT,
        BathHalfCount      INT,
        Bath3qtrCount      INT,
        BathFullCount      INT,
        FpSingleStory      INT,
        FpMultiStory       INT,
        FpFreestanding     INT,
        FpAdditional       INT,
        YrBuilt            INT,
        YrRenovated        INT,
        PcntComplete       INT,
        Obsolescence       INT,
        PcntNetCondition   INT,
        Condition          INT,
        AddnlCost          INT
    );
"""

In [66]:
cursor = conn.cursor()

In [67]:
cursor.execute(CREATE_BUILDINGS_TABLE_QUERY)

In [68]:
from data_collection import collect_building_data
buildings_zip_file, buildings_csv_file = collect_building_data()

In [69]:
next(buildings_csv_file)

'"Major","Minor","BldgNbr","NbrLivingUnits","Address","BuildingNumber","Fraction","DirectionPrefix","StreetName","StreetType","DirectionSuffix","ZipCode","Stories","BldgGrade","BldgGradeVar","SqFt1stFloor","SqFtHalfFloor","SqFt2ndFloor","SqFtUpperFloor","SqFtUnfinFull","SqFtUnfinHalf","SqFtTotLiving","SqFtTotBasement","SqFtFinBasement","FinBasementGrade","SqFtGarageBasement","SqFtGarageAttached","DaylightBasement","SqFtOpenPorch","SqFtEnclosedPorch","SqFtDeck","HeatSystem","HeatSource","BrickStone","ViewUtilization","Bedrooms","BathHalfCount","Bath3qtrCount","BathFullCount","FpSingleStory","FpMultiStory","FpFreestanding","FpAdditional","YrBuilt","YrRenovated","PcntComplete","Obsolescence","PcntNetCondition","Condition","AddnlCost"\n'

In [70]:
COPY_BUILDINGS_QUERY = """COPY buildings FROM STDIN WITH (FORMAT CSV)"""

In [71]:
cursor.copy_expert(COPY_BUILDINGS_QUERY, buildings_csv_file)

InvalidTextRepresentation: invalid input syntax for integer: "1.5"
CONTEXT:  COPY buildings, line 2, column stories: "1.5"


In [72]:
# ok, looks like we are going to have to deal with different types of "numbers"

In [73]:
CREATE_BUILDINGS_TABLE_QUERY = """
    DROP TABLE IF EXISTS buildings;
    CREATE TABLE BUILDINGS (
        Major              CHAR(6),
        Minor              CHAR(4),
        BldgNbr            INT,
        NbrLivingUnits     INT,
        Address            TEXT,
        BuildingNumber     CHAR(5),
        Fraction           CHAR(3),
        DirectionPrefix    CHAR(2),
        StreetName         CHAR(25),
        StreetType         CHAR(6),
        DirectionSuffix    CHAR(2),
        ZipCode            CHAR(10),
        Stories            REAL,
        BldgGrade          INT,
        BldgGradeVar       INT,
        SqFt1stFloor       INT,
        SqFtHalfFloor      INT,
        SqFt2ndFloor       INT,
        SqFtUpperFloor     INT,
        SqFtUnfinFull      INT,
        SqFtUnfinHalf      INT,
        SqFtTotLiving      INT,
        SqFtTotBasement    INT,
        SqFtFinBasement    INT,
        FinBasementGrade   INT,
        SqFtGarageBasement INT,
        SqFtGarageAttached INT,
        DaylightBasement   CHAR(1),
        SqFtOpenPorch      INT,
        SqFtEnclosedPorch  INT,
        SqFtDeck           INT,
        HeatSystem         INT,
        HeatSource         INT,
        BrickStone         INT,
        ViewUtilization    CHAR(1),
        Bedrooms           INT,
        BathHalfCount      INT,
        Bath3qtrCount      INT,
        BathFullCount      INT,
        FpSingleStory      INT,
        FpMultiStory       INT,
        FpFreestanding     INT,
        FpAdditional       INT,
        YrBuilt            INT,
        YrRenovated        INT,
        PcntComplete       INT,
        Obsolescence       INT,
        PcntNetCondition   INT,
        Condition          INT,
        AddnlCost          INT
    );
"""

In [74]:
conn.close()

In [75]:
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()
cursor.execute(CREATE_BUILDINGS_TABLE_QUERY)

In [76]:
buildings_zip_file, buildings_csv_file = collect_building_data()
next(buildings_csv_file)

'"Major","Minor","BldgNbr","NbrLivingUnits","Address","BuildingNumber","Fraction","DirectionPrefix","StreetName","StreetType","DirectionSuffix","ZipCode","Stories","BldgGrade","BldgGradeVar","SqFt1stFloor","SqFtHalfFloor","SqFt2ndFloor","SqFtUpperFloor","SqFtUnfinFull","SqFtUnfinHalf","SqFtTotLiving","SqFtTotBasement","SqFtFinBasement","FinBasementGrade","SqFtGarageBasement","SqFtGarageAttached","DaylightBasement","SqFtOpenPorch","SqFtEnclosedPorch","SqFtDeck","HeatSystem","HeatSource","BrickStone","ViewUtilization","Bedrooms","BathHalfCount","Bath3qtrCount","BathFullCount","FpSingleStory","FpMultiStory","FpFreestanding","FpAdditional","YrBuilt","YrRenovated","PcntComplete","Obsolescence","PcntNetCondition","Condition","AddnlCost"\n'

In [77]:
cursor.copy_expert(COPY_BUILDINGS_QUERY, buildings_csv_file)

In [78]:
conn.commit()

In [79]:
pd.read_sql_query("SELECT * FROM buildings LIMIT 5;", conn)

,major,minor,bldgnbr,nbrlivingunits,address,buildingnumber,fraction,directionprefix,streetname,streettype,...,fpmultistory,fpfreestanding,fpadditional,yrbuilt,yrrenovated,pcntcomplete,obsolescence,pcntnetcondition,condition,addnlcost
0,012000,0032,1,1,830 8TH AVE S 98033,830,,,8TH,AVE,...,0,0,0,1981,0,0,0,0,3,0
1,012000,0260,1,1,341 8TH ST S 98033,341,,,8TH,ST,...,0,0,0,1928,2007,0,0,0,3,0
2,012000,0262,1,1,712 5TH AVE S 98033,712,,,5TH,AVE,...,0,0,0,2010,0,0,0,0,3,0
3,012005,9035,1,1,39525 AUBURN-ENUMCLAW RD SE 98092,39525,,,AUBURN-ENUMCLAW,RD,...,0,0,0,1917,0,0,0,0,1,0
4,012006,9079,1,1,39007 272ND AVE SE 98022,39007,,,272ND,AVE,...,0,0,0,1990,0,0,0,0,4,0


In [80]:
conn.close()

In [81]:
# run through the whole buildings flow
from data_collection import collect_building_data
buildings_zip_file, buildings_csv_file = collect_building_data()

In [82]:
from sql_utils import create_database, create_buildings_table, copy_csv_to_buildings_table
create_database()

In [83]:
conn = psycopg2.connect(dbname="housing_data")

In [84]:
create_buildings_table(conn)

In [85]:
copy_csv_to_buildings_table(conn, buildings_csv_file)

In [86]:
buildings_zip_file.close()
buildings_csv_file.close()
conn.close()

In [87]:
# test whether it worked
conn = psycopg2.connect(dbname="housing_data")
pd.read_sql_query("SELECT * FROM buildings LIMIT 5;", conn)

,major,minor,bldgnbr,nbrlivingunits,address,buildingnumber,fraction,directionprefix,streetname,streettype,...,fpmultistory,fpfreestanding,fpadditional,yrbuilt,yrrenovated,pcntcomplete,obsolescence,pcntnetcondition,condition,addnlcost
0,012000,0032,1,1,830 8TH AVE S 98033,830,,,8TH,AVE,...,0,0,0,1981,0,0,0,0,3,0
1,012000,0260,1,1,341 8TH ST S 98033,341,,,8TH,ST,...,0,0,0,1928,2007,0,0,0,3,0
2,012000,0262,1,1,712 5TH AVE S 98033,712,,,5TH,AVE,...,0,0,0,2010,0,0,0,0,3,0
3,012005,9035,1,1,39525 AUBURN-ENUMCLAW RD SE 98092,39525,,,AUBURN-ENUMCLAW,RD,...,0,0,0,1917,0,0,0,0,1,0
4,012006,9079,1,1,39007 272ND AVE SE 98022,39007,,,272ND,AVE,...,0,0,0,1990,0,0,0,0,4,0


In [88]:
# last one: parcels

In [89]:
CREATE_PARCELS_TABLE_QUERY = """
    DROP TABLE IF EXISTS parcels;
    CREATE TABLE parcels (
        Major                  CHAR(6),
        Minor                  CHAR(4),
        PropName               CHAR(80),
        PlatName               CHAR(50),
        PlatLot                CHAR(14),
        PlatBlock              CHAR(7),
        Range                  INT,
        Township               INT,
        Section                INT,
        QuarterSection         CHAR(2),
        PropType               CHAR(1),
        Area                   CHAR(3),
        SubArea                CHAR(3),
        SpecArea               CHAR(3),
        SpecSubArea            CHAR(3),
        DistrictName           CHAR(80),
        LevyCode               CHAR(4),
        CurrentZoning          CHAR(50),
        HBUAsIfVacant          INT,
        HBUAsImproved          INT,
        PresentUse             INT,
        SqFtLot                INT,
        WaterSystem            INT,
        SewerSystem            INT,
        Access                 INT,
        Topography             INT,
        StreetSurface          INT,
        RestrictiveSzShape     INT,
        InadequateParking      INT,
        PcntUnusable           INT,
        Unbuildable            INT, -- "Unbuildable" is not in the DOC
        MtRainier              INT,
        Olympics               INT,
        Cascades               INT,
        Territorial            INT,
        SeattleSkyline         INT,
        PugetSound             INT,
        LakeWashington         INT,
        LakeSammamish          INT,
        SmallLakeRiverCreek    INT,
        OtherView              INT,
        WfntLocation           INT,
        WfntFootage            INT,
        WfntBank               INT,
        WfntPoorQuality        INT,
        WfntRestrictedAccess   INT,
        WfntAccessRights       CHAR(1),
        WfntProximityInfluence CHAR(1),
        TidelandShoreland      INT,
        LotDepthFactor         INT,
        TrafficNoise           INT,
        AirportNoise           INT,
        PowerLines             CHAR(1),
        OtherNuisances         CHAR(1),
        NbrBldgSites           INT,
        Contamination          INT,
        DNRLease               CHAR(1),
        AdjacentGolfFairway    CHAR(1),
        AdjacentGreenbelt      CHAR(1),
        -- "Common Property" is listed here in the DOC, but isn't in the CSV
        HistoricSite           INT,
        CurrentUseDesignation  INT,
        NativeGrowthProtEsmt   CHAR(1),
        Easements              CHAR(1),
        OtherDesignation       CHAR(1),
        DeedRestrictions       CHAR(1),
        DevelopmentRightsPurch CHAR(1),
        CoalMineHazard         CHAR(1),
        CriticalDrainage       CHAR(1),
        ErosionHazard          CHAR(1),
        LandfillBuffer         CHAR(1),
        HundredYrFloodPlain    CHAR(1),
        SeismicHazard          CHAR(1),
        LandslideHazard        CHAR(1),
        SteepSlopeHazard       CHAR(1),
        Stream                 CHAR(1),
        Wetland                CHAR(1),
        SpeciesOfConcern       CHAR(1),
        SensitiveAreaTract     CHAR(1),
        WaterProblems          CHAR(1),
        TranspConcurrency      CHAR(1),
        OtherProblems          CHAR(1)
    );
"""

In [90]:
from data_collection import collect_parcels_data
parcels_zip_file, parcels_csv_file = collect_parcels_data()

In [91]:
conn = psycopg2.connect(dbname="housing_data")

In [93]:
cursor = conn.cursor()
cursor.execute(CREATE_PARCELS_TABLE_QUERY)

In [94]:
COPY_PARCELS_QUERY = """COPY buildings FROM STDIN WITH (FORMAT CSV)"""

In [95]:
next(parcels_csv_file)

'"Major","Minor","PropName","PlatName","PlatLot","PlatBlock","Range","Township","Section","QuarterSection","PropType","Area","SubArea","SpecArea","SpecSubArea","DistrictName","LevyCode","CurrentZoning","HBUAsIfVacant","HBUAsImproved","PresentUse","SqFtLot","WaterSystem","SewerSystem","Access","Topography","StreetSurface","RestrictiveSzShape","InadequateParking","PcntUnusable","Unbuildable","MtRainier","Olympics","Cascades","Territorial","SeattleSkyline","PugetSound","LakeWashington","LakeSammamish","SmallLakeRiverCreek","OtherView","WfntLocation","WfntFootage","WfntBank","WfntPoorQuality","WfntRestrictedAccess","WfntAccessRights","WfntProximityInfluence","TidelandShoreland","LotDepthFactor","TrafficNoise","AirportNoise","PowerLines","OtherNuisances","NbrBldgSites","Contamination","DNRLease","AdjacentGolfFairway","AdjacentGreenbelt","HistoricSite","CurrentUseDesignation","NativeGrowthProtEsmt","Easements","OtherDesignation","DeedRestrictions","DevelopmentRightsPurch","CoalMineHazard","C

In [96]:
cursor.copy_expert(COPY_PARCELS_QUERY, parcels_csv_file)

BadCopyFileFormat: extra data after last expected column
CONTEXT:  COPY buildings, line 1: ""052307","9017"," ","","              ","       ","7","23","5","  ","M","0","0","","","KING COUNTY",..."


In [98]:
conn.close()

In [99]:
parcels_zip_file, parcels_csv_file = collect_parcels_data()

In [100]:
from csv import DictReader

In [101]:
parcels_reader = DictReader(parcels_csv_file)

In [103]:
len(parcels_reader.fieldnames)

81

In [104]:
first = next(parcels_reader)

In [105]:
first

{'Major': '052307',
 'Minor': '9017',
 'PropName': ' ',
 'PlatName': '',
 'PlatLot': '              ',
 'PlatBlock': '       ',
 'Range': '7',
 'Township': '23',
 'Section': '5',
 'QuarterSection': '  ',
 'PropType': 'M',
 'Area': '0',
 'SubArea': '0',
 'SpecArea': '',
 'SpecSubArea': '',
 'DistrictName': 'KING COUNTY',
 'LevyCode': '6982',
 'CurrentZoning': ' ',
 'HBUAsIfVacant': '0',
 'HBUAsImproved': '0',
 'PresentUse': '0',
 'SqFtLot': '0',
 'WaterSystem': '0',
 'SewerSystem': '0',
 'Access': '0',
 'Topography': '0',
 'StreetSurface': '0',
 'RestrictiveSzShape': '0',
 'InadequateParking': '0',
 'PcntUnusable': '0',
 'Unbuildable': 'False',
 'MtRainier': '0',
 'Olympics': '0',
 'Cascades': '0',
 'Territorial': '0',
 'SeattleSkyline': '0',
 'PugetSound': '0',
 'LakeWashington': '0',
 'LakeSammamish': '0',
 'SmallLakeRiverCreek': '0',
 'OtherView': '0',
 'WfntLocation': '0',
 'WfntFootage': '0',
 'WfntBank': '0',
 'WfntPoorQuality': '0',
 'WfntRestrictedAccess': '0',
 'WfntAccessRight

In [106]:
len(first)

81

In [107]:
from csv import reader

In [108]:
parcels_zip_file, parcels_csv_file = collect_parcels_data()

In [109]:
parcels_reader = reader(parcels_csv_file)

In [110]:
header = next(parcels_reader)

In [111]:
header

['Major',
 'Minor',
 'PropName',
 'PlatName',
 'PlatLot',
 'PlatBlock',
 'Range',
 'Township',
 'Section',
 'QuarterSection',
 'PropType',
 'Area',
 'SubArea',
 'SpecArea',
 'SpecSubArea',
 'DistrictName',
 'LevyCode',
 'CurrentZoning',
 'HBUAsIfVacant',
 'HBUAsImproved',
 'PresentUse',
 'SqFtLot',
 'WaterSystem',
 'SewerSystem',
 'Access',
 'Topography',
 'StreetSurface',
 'RestrictiveSzShape',
 'InadequateParking',
 'PcntUnusable',
 'Unbuildable',
 'MtRainier',
 'Olympics',
 'Cascades',
 'Territorial',
 'SeattleSkyline',
 'PugetSound',
 'LakeWashington',
 'LakeSammamish',
 'SmallLakeRiverCreek',
 'OtherView',
 'WfntLocation',
 'WfntFootage',
 'WfntBank',
 'WfntPoorQuality',
 'WfntRestrictedAccess',
 'WfntAccessRights',
 'WfntProximityInfluence',
 'TidelandShoreland',
 'LotDepthFactor',
 'TrafficNoise',
 'AirportNoise',
 'PowerLines',
 'OtherNuisances',
 'NbrBldgSites',
 'Contamination',
 'DNRLease',
 'AdjacentGolfFairway',
 'AdjacentGreenbelt',
 'HistoricSite',
 'CurrentUseDesignatio

In [112]:
len(header)

81

In [113]:
first = next(parcels_reader)

In [114]:
first

['052307',
 '9017',
 ' ',
 '',
 '              ',
 '       ',
 '7',
 '23',
 '5',
 '  ',
 'M',
 '0',
 '0',
 '',
 '',
 'KING COUNTY',
 '6982',
 ' ',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'False',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'N',
 'N',
 '0',
 '0',
 '0',
 '0',
 'N',
 'N',
 '0',
 '0',
 'N',
 'N',
 'N',
 '0',
 '0',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N']

In [115]:
len(first)

81

In [116]:
# I can't see anything wrong with the first row...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [117]:
parcels_zip_file, parcels_csv_file = collect_parcels_data()

In [118]:
header = parcels_csv_file.readline()
header

'"Major","Minor","PropName","PlatName","PlatLot","PlatBlock","Range","Township","Section","QuarterSection","PropType","Area","SubArea","SpecArea","SpecSubArea","DistrictName","LevyCode","CurrentZoning","HBUAsIfVacant","HBUAsImproved","PresentUse","SqFtLot","WaterSystem","SewerSystem","Access","Topography","StreetSurface","RestrictiveSzShape","InadequateParking","PcntUnusable","Unbuildable","MtRainier","Olympics","Cascades","Territorial","SeattleSkyline","PugetSound","LakeWashington","LakeSammamish","SmallLakeRiverCreek","OtherView","WfntLocation","WfntFootage","WfntBank","WfntPoorQuality","WfntRestrictedAccess","WfntAccessRights","WfntProximityInfluence","TidelandShoreland","LotDepthFactor","TrafficNoise","AirportNoise","PowerLines","OtherNuisances","NbrBldgSites","Contamination","DNRLease","AdjacentGolfFairway","AdjacentGreenbelt","HistoricSite","CurrentUseDesignation","NativeGrowthProtEsmt","Easements","OtherDesignation","DeedRestrictions","DevelopmentRightsPurch","CoalMineHazard","C

In [119]:
first = parcels_csv_file.readline()
first

'"052307","9017"," ","","              ","       ","7","23","5","  ","M","0","0","","","KING COUNTY","6982"," ","0","0","0","0","0","0","0","0","0","0","0","0","False","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","N","N","0","0","0","0","N","N","0","0","N","N","N","0","0","N","N","N","N","N","N","N","N","N","N","N","N","N","N","N","N","N","N","N","N"\n'

In [120]:
first_list = first.split(",")
first_list

['"052307"',
 '"9017"',
 '" "',
 '""',
 '"              "',
 '"       "',
 '"7"',
 '"23"',
 '"5"',
 '"  "',
 '"M"',
 '"0"',
 '"0"',
 '""',
 '""',
 '"KING COUNTY"',
 '"6982"',
 '" "',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"False"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"N"',
 '"N"',
 '"0"',
 '"0"',
 '"0"',
 '"0"',
 '"N"',
 '"N"',
 '"0"',
 '"0"',
 '"N"',
 '"N"',
 '"N"',
 '"0"',
 '"0"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"',
 '"N"\n']

In [121]:
len(first_list)

81

In [122]:
# just in case, try removing the comments from the SQL
CREATE_PARCELS_TABLE_QUERY = """
    DROP TABLE IF EXISTS parcels;
    CREATE TABLE parcels (
        Major                  CHAR(6),
        Minor                  CHAR(4),
        PropName               CHAR(80),
        PlatName               CHAR(50),
        PlatLot                CHAR(14),
        PlatBlock              CHAR(7),
        Range                  INT,
        Township               INT,
        Section                INT,
        QuarterSection         CHAR(2),
        PropType               CHAR(1),
        Area                   CHAR(3),
        SubArea                CHAR(3),
        SpecArea               CHAR(3),
        SpecSubArea            CHAR(3),
        DistrictName           CHAR(80),
        LevyCode               CHAR(4),
        CurrentZoning          CHAR(50),
        HBUAsIfVacant          INT,
        HBUAsImproved          INT,
        PresentUse             INT,
        SqFtLot                INT,
        WaterSystem            INT,
        SewerSystem            INT,
        Access                 INT,
        Topography             INT,
        StreetSurface          INT,
        RestrictiveSzShape     INT,
        InadequateParking      INT,
        PcntUnusable           INT,
        Unbuildable            INT,
        MtRainier              INT,
        Olympics               INT,
        Cascades               INT,
        Territorial            INT,
        SeattleSkyline         INT,
        PugetSound             INT,
        LakeWashington         INT,
        LakeSammamish          INT,
        SmallLakeRiverCreek    INT,
        OtherView              INT,
        WfntLocation           INT,
        WfntFootage            INT,
        WfntBank               INT,
        WfntPoorQuality        INT,
        WfntRestrictedAccess   INT,
        WfntAccessRights       CHAR(1),
        WfntProximityInfluence CHAR(1),
        TidelandShoreland      INT,
        LotDepthFactor         INT,
        TrafficNoise           INT,
        AirportNoise           INT,
        PowerLines             CHAR(1),
        OtherNuisances         CHAR(1),
        NbrBldgSites           INT,
        Contamination          INT,
        DNRLease               CHAR(1),
        AdjacentGolfFairway    CHAR(1),
        AdjacentGreenbelt      CHAR(1),
        HistoricSite           INT,
        CurrentUseDesignation  INT,
        NativeGrowthProtEsmt   CHAR(1),
        Easements              CHAR(1),
        OtherDesignation       CHAR(1),
        DeedRestrictions       CHAR(1),
        DevelopmentRightsPurch CHAR(1),
        CoalMineHazard         CHAR(1),
        CriticalDrainage       CHAR(1),
        ErosionHazard          CHAR(1),
        LandfillBuffer         CHAR(1),
        HundredYrFloodPlain    CHAR(1),
        SeismicHazard          CHAR(1),
        LandslideHazard        CHAR(1),
        SteepSlopeHazard       CHAR(1),
        Stream                 CHAR(1),
        Wetland                CHAR(1),
        SpeciesOfConcern       CHAR(1),
        SensitiveAreaTract     CHAR(1),
        WaterProblems          CHAR(1),
        TranspConcurrency      CHAR(1),
        OtherProblems          CHAR(1)
    );
"""

In [123]:
conn.close()

In [124]:
parcels_zip_file, parcels_csv_file = collect_parcels_data()
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()
cursor.execute(CREATE_PARCELS_TABLE_QUERY)
# dangit, this was a copypasta mistake
COPY_PARCELS_QUERY = """COPY parcels FROM STDIN WITH (FORMAT CSV)"""
next(parcels_csv_file)

'"Major","Minor","PropName","PlatName","PlatLot","PlatBlock","Range","Township","Section","QuarterSection","PropType","Area","SubArea","SpecArea","SpecSubArea","DistrictName","LevyCode","CurrentZoning","HBUAsIfVacant","HBUAsImproved","PresentUse","SqFtLot","WaterSystem","SewerSystem","Access","Topography","StreetSurface","RestrictiveSzShape","InadequateParking","PcntUnusable","Unbuildable","MtRainier","Olympics","Cascades","Territorial","SeattleSkyline","PugetSound","LakeWashington","LakeSammamish","SmallLakeRiverCreek","OtherView","WfntLocation","WfntFootage","WfntBank","WfntPoorQuality","WfntRestrictedAccess","WfntAccessRights","WfntProximityInfluence","TidelandShoreland","LotDepthFactor","TrafficNoise","AirportNoise","PowerLines","OtherNuisances","NbrBldgSites","Contamination","DNRLease","AdjacentGolfFairway","AdjacentGreenbelt","HistoricSite","CurrentUseDesignation","NativeGrowthProtEsmt","Easements","OtherDesignation","DeedRestrictions","DevelopmentRightsPurch","CoalMineHazard","C

In [125]:
cursor.copy_expert(COPY_PARCELS_QUERY, parcels_csv_file)

InvalidTextRepresentation: invalid input syntax for integer: "False"
CONTEXT:  COPY parcels, line 1, column unbuildable: "False"


In [126]:
CREATE_PARCELS_TABLE_QUERY = """
    DROP TABLE IF EXISTS parcels;
    CREATE TABLE parcels (
        Major                  CHAR(6),
        Minor                  CHAR(4),
        PropName               CHAR(80),
        PlatName               CHAR(50),
        PlatLot                CHAR(14),
        PlatBlock              CHAR(7),
        Range                  INT,
        Township               INT,
        Section                INT,
        QuarterSection         CHAR(2),
        PropType               CHAR(1),
        Area                   CHAR(3),
        SubArea                CHAR(3),
        SpecArea               CHAR(3),
        SpecSubArea            CHAR(3),
        DistrictName           CHAR(80),
        LevyCode               CHAR(4),
        CurrentZoning          CHAR(50),
        HBUAsIfVacant          INT,
        HBUAsImproved          INT,
        PresentUse             INT,
        SqFtLot                INT,
        WaterSystem            INT,
        SewerSystem            INT,
        Access                 INT,
        Topography             INT,
        StreetSurface          INT,
        RestrictiveSzShape     INT,
        InadequateParking      INT,
        PcntUnusable           INT,
        Unbuildable            CHAR(5), -- "Unbuildable" is not in the DOC
        MtRainier              INT,
        Olympics               INT,
        Cascades               INT,
        Territorial            INT,
        SeattleSkyline         INT,
        PugetSound             INT,
        LakeWashington         INT,
        LakeSammamish          INT,
        SmallLakeRiverCreek    INT,
        OtherView              INT,
        WfntLocation           INT,
        WfntFootage            INT,
        WfntBank               INT,
        WfntPoorQuality        INT,
        WfntRestrictedAccess   INT,
        WfntAccessRights       CHAR(1),
        WfntProximityInfluence CHAR(1),
        TidelandShoreland      INT,
        LotDepthFactor         INT,
        TrafficNoise           INT,
        AirportNoise           INT,
        PowerLines             CHAR(1),
        OtherNuisances         CHAR(1),
        NbrBldgSites           INT,
        Contamination          INT,
        DNRLease               CHAR(1),
        AdjacentGolfFairway    CHAR(1),
        AdjacentGreenbelt      CHAR(1),
        -- "Common Property" is listed here in the DOC, but isn't in the CSV
        HistoricSite           INT,
        CurrentUseDesignation  INT,
        NativeGrowthProtEsmt   CHAR(1),
        Easements              CHAR(1),
        OtherDesignation       CHAR(1),
        DeedRestrictions       CHAR(1),
        DevelopmentRightsPurch CHAR(1),
        CoalMineHazard         CHAR(1),
        CriticalDrainage       CHAR(1),
        ErosionHazard          CHAR(1),
        LandfillBuffer         CHAR(1),
        HundredYrFloodPlain    CHAR(1),
        SeismicHazard          CHAR(1),
        LandslideHazard        CHAR(1),
        SteepSlopeHazard       CHAR(1),
        Stream                 CHAR(1),
        Wetland                CHAR(1),
        SpeciesOfConcern       CHAR(1),
        SensitiveAreaTract     CHAR(1),
        WaterProblems          CHAR(1),
        TranspConcurrency      CHAR(1),
        OtherProblems          CHAR(1)
    );
"""

In [127]:
conn.close()
parcels_zip_file, parcels_csv_file = collect_parcels_data()
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()
cursor.execute(CREATE_PARCELS_TABLE_QUERY)
COPY_PARCELS_QUERY = """COPY parcels FROM STDIN WITH (FORMAT CSV)"""
next(parcels_csv_file)

'"Major","Minor","PropName","PlatName","PlatLot","PlatBlock","Range","Township","Section","QuarterSection","PropType","Area","SubArea","SpecArea","SpecSubArea","DistrictName","LevyCode","CurrentZoning","HBUAsIfVacant","HBUAsImproved","PresentUse","SqFtLot","WaterSystem","SewerSystem","Access","Topography","StreetSurface","RestrictiveSzShape","InadequateParking","PcntUnusable","Unbuildable","MtRainier","Olympics","Cascades","Territorial","SeattleSkyline","PugetSound","LakeWashington","LakeSammamish","SmallLakeRiverCreek","OtherView","WfntLocation","WfntFootage","WfntBank","WfntPoorQuality","WfntRestrictedAccess","WfntAccessRights","WfntProximityInfluence","TidelandShoreland","LotDepthFactor","TrafficNoise","AirportNoise","PowerLines","OtherNuisances","NbrBldgSites","Contamination","DNRLease","AdjacentGolfFairway","AdjacentGreenbelt","HistoricSite","CurrentUseDesignation","NativeGrowthProtEsmt","Easements","OtherDesignation","DeedRestrictions","DevelopmentRightsPurch","CoalMineHazard","C

In [128]:
cursor.copy_expert(COPY_PARCELS_QUERY, parcels_csv_file)

QueryCanceled: COPY from stdin failed: error in .read() call: UnicodeDecodeError 'utf-8' codec can't decode byte 0xe9 in position 3476: invalid continuation byte
CONTEXT:  COPY parcels, line 83265


In [129]:
# this file doesn't have valid utf-8 encoding, ya gotta be kidding me

In [130]:
parcels_zip_file, parcels_csv_file = collect_parcels_data()

In [131]:
# ^ specified a different encoding scheme in the open_csv function
conn.close()

In [132]:
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()
cursor.execute(CREATE_PARCELS_TABLE_QUERY)
COPY_PARCELS_QUERY = """COPY parcels FROM STDIN WITH (FORMAT CSV)"""
next(parcels_csv_file)

'"Major","Minor","PropName","PlatName","PlatLot","PlatBlock","Range","Township","Section","QuarterSection","PropType","Area","SubArea","SpecArea","SpecSubArea","DistrictName","LevyCode","CurrentZoning","HBUAsIfVacant","HBUAsImproved","PresentUse","SqFtLot","WaterSystem","SewerSystem","Access","Topography","StreetSurface","RestrictiveSzShape","InadequateParking","PcntUnusable","Unbuildable","MtRainier","Olympics","Cascades","Territorial","SeattleSkyline","PugetSound","LakeWashington","LakeSammamish","SmallLakeRiverCreek","OtherView","WfntLocation","WfntFootage","WfntBank","WfntPoorQuality","WfntRestrictedAccess","WfntAccessRights","WfntProximityInfluence","TidelandShoreland","LotDepthFactor","TrafficNoise","AirportNoise","PowerLines","OtherNuisances","NbrBldgSites","Contamination","DNRLease","AdjacentGolfFairway","AdjacentGreenbelt","HistoricSite","CurrentUseDesignation","NativeGrowthProtEsmt","Easements","OtherDesignation","DeedRestrictions","DevelopmentRightsPurch","CoalMineHazard","C

In [133]:
cursor.copy_expert(COPY_PARCELS_QUERY, parcels_csv_file)

In [134]:
conn.commit()

In [135]:
conn.close()

In [136]:
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()

In [137]:
pd.read_sql_query("SELECT * FROM parcels LIMIT 5;", conn)

,major,minor,propname,platname,platlot,platblock,range,township,section,quartersection,...,seismichazard,landslidehazard,steepslopehazard,stream,wetland,speciesofconcern,sensitiveareatract,waterproblems,transpconcurrency,otherproblems
0,052307,9017,...,...,,,7,23,5,,...,N,N,N,N,N,N,N,N,N,N
1,245000,0086,...,FAILORS ENATAI GARDENS ADD ...,17,1,0,0,0,,...,N,N,N,N,N,N,N,N,N,N
2,377920,0165,...,JUNCTION ADD ...,28B & 29 TO 32,,4,23,13,SW,...,N,N,N,N,N,N,N,N,N,N
3,277160,5275,...,GILMANS ADD BLKS 149 THRU 205 TR A-E VZ ...,,,3,25,14,NE,...,N,N,N,N,N,N,N,N,N,N
4,766620,3435,...,SEATTLE TIDE LDS BL 01-376 ...,POR,248,4,24,8,NE,...,N,N,N,N,N,N,N,N,N,N


In [138]:
pd.read_sql_query("SELECT * FROM parcels LIMIT 5 OFFSET 83265;", conn)

,major,minor,propname,platname,platlot,platblock,range,township,section,quartersection,...,seismichazard,landslidehazard,steepslopehazard,stream,wetland,speciesofconcern,sensitiveareatract,waterproblems,transpconcurrency,otherproblems
0,289573,0060,...,GREENBRIAR ...,6,,6,25,32,SE,...,N,N,N,N,N,N,N,N,N,N
1,062981,0650,...,BEAVERDAM DIV NO. 02 ...,65,,6,25,35,SW,...,N,N,N,N,N,N,N,N,N,N
2,637450,0090,...,OLSON LAND COS 8TH ADD TRS ...,PORTION,18,3,24,10,SW,...,N,N,N,N,N,N,N,N,N,N
3,510040,1160,...,MAPLE LEAF TO GREEN LAKE CIRCLE POR OF ...,2-3 AC 1,15,4,26,32,SE,...,N,N,N,N,N,N,N,N,N,N
4,375510,0340,...,JUANITA ESTATES DIV NO. 03 ...,34,,5,26,19,SW,...,N,N,N,N,N,N,N,N,N,N


In [139]:
# seems like the less-strict encoding is fine!

In [140]:
conn.close()